[Initial Environment]

In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import sys
import os
sys.path.append(os.path.join(".."))
import libPython.core.util

In [1]:
import logging
logging.basicConfig( 
        level=logging.DEBUG,
        # level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(module)s.%(funcName)s():%(lineno)d %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
        )
logging.info("this is message")

2021-12-16 20:44:40 [INFO] <ipython-input-1-f5c87b5c617d>.<module>():8 this is message


In [4]:
import logging

logging.basicConfig( 
        level=logging.DEBUG,
        # level=logging.INFO,
        format=None,
        datefmt=None
        )
logging.info("this is message")

this is message


In [15]:
l = [ 9,9,1,2,4,4,5]
print(l)
s=set(l)
print(s)
l2=list(s)
print(l2[-1])

[9, 9, 1, 2, 4, 4, 5]
{1, 2, 4, 5, 9}
9


In [2]:
from FinMind.data import DataLoader

api = DataLoader()
# api.login_by_token(api_token='token')
# api.login(user_id='user_id',password='password')
df = api.taiwan_daily_short_sale_balances(
    stock_id="2330",
    start_date='2020-04-01',
    end_date='2020-04-12'
)

In [7]:
from finlab import data
data.get('price:收盤價')

ModuleNotFoundError: No module named 'finlab'